In [1]:
import numpy as np

In [2]:
import sys
import os

from PIL import Image
import matplotlib.pyplot as plt

import copy

In [3]:
dataset = 'CIFAR100'
num_cluster = 10


T =5 
max_length = 200
Sparse = 20

out_path = f'output_iid/{dataset}_{num_cluster}_T{T}_length{max_length}_Sparse{Sparse}/'


In [4]:
out_name =f'{dataset}_{num_cluster}_T{T}_length{max_length}_Sparse{Sparse}_df'

In [5]:
files = os.listdir(out_path)

In [6]:
if '.ipynb_checkpoints' in files:
    files.remove('.ipynb_checkpoints')
if 'data' in files:
    files.remove('data')

In [7]:
files

['1984.npy',
 '3265.npy',
 '4954.npy',
 '5262.npy',
 '5718.npy',
 '6315.npy',
 '8229.npy',
 '9766.npy',
 '9960.npy']

In [8]:
performance = np.zeros((len(files),28,T))

In [9]:
active_rate = [0.,0.2,0.4,0.6,0.8,1.]

In [10]:
len_rate =  len(active_rate)

In [11]:
legend0 = [f'Nothing ({i})' for i in active_rate]
legend1 = [f'Ours ({i})' for i in active_rate]
legend2 = [f'Uniform w/o feedback ({i}) ' for i in active_rate]
legend3 = [f'Uniform+Sampling Sketch ({i})' for i in active_rate]
# eandom
legend4 = [f'Uniform+Random Sketch']
legend5 = [f'Random W.+MAP Sketch ({i})' for i in active_rate]
legend6 = [f'Random W.+Sampling Sketch ({i})' for i in active_rate]
all_legend = ['global','Maximum Diveristy']+legend0+legend1+legend2+legend3+legend4+legend5+legend6+ ['Random Selection']

In [12]:
performance = np.zeros((len(files),len(all_legend),T))

In [13]:
for i in range(len(files)):
    all_resluts = np.load(out_path+files[i],allow_pickle=True)
    performance[i,0,:] = all_resluts[0] #global
    performance[i,1,:] = all_resluts[1] #
    # performance[i,2:2+len_rate,:] =0 #np.mean(all_resluts[2]) #ours
    performance[i,2+len_rate:2+2*len_rate,:] =np.mean(all_resluts[3][:,0]) # uniform with feedback
    performance[i,2+2*len_rate:2+3*len_rate,:] =np.mean(all_resluts[4][:,0]) # uniform without feedback
    performance[i,2+3*len_rate:2+4*len_rate,:] =np.mean(all_resluts[5][:,0]) # uniform with feedback sampling
    performance[i,2+4*len_rate,:] = np.mean(all_resluts[6][:,0],axis=0) #random submatrix
    performance[i,3+4*len_rate:3+5*len_rate,:] = np.mean(all_resluts[7][:,0]) #randw MAP F
    performance[i,3+5*len_rate:3+6*len_rate,:] = np.mean(all_resluts[8][:,0]) # rand w  SAMPLING f
    performance[i,3+6*len_rate,:] = all_resluts[-4]  #random
args = all_resluts[-1]

In [14]:
num_method = performance.shape[1]
Summaray = np.zeros((num_method,T))
Summaray_var = np.zeros((num_method,T))
for j in range(num_method):
    for i in range(T):
        index = np.where(performance[:,j,i]>0)[0]
        Summaray[j,i] = -np.mean(performance[index,j,i]-performance[index,0,i])
        Summaray_var[j,i] = np.std(performance[index,j,i]-performance[index,0,i])

/home/xiwenc/.local/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/xiwenc/.local/lib/python3.9/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/xiwenc/.local/lib/python3.9/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/xiwenc/.local/lib/python3.9/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/home/xiwenc/.local/lib/python3.9/site-packages/numpy/core/_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [15]:
import pandas as pd

In [16]:
row = all_legend
df1 = pd.DataFrame(Summaray, columns =[f'T={i}' for i in range(1,T+1)], index =row)

In [17]:
# df1

In [18]:
df1.drop(legend0, inplace = True)

In [19]:
# df1.to_csv('output/'+out_name+'1.csv')

In [20]:
df1

,T=1,T=2,T=3,T=4,T=5
global,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000
Maximum Diveristy,40.204123,75.974354,108.635176,140.323815,167.845798
Ours (0.0),15.443813,27.955487,37.468134,47.638748,58.229404
Ours (0.2),15.443813,28.267592,37.787798,46.696443,57.053835
Ours (0.4),15.443813,28.482393,37.428522,46.929779,56.676632
Ours (0.6),15.443813,27.895310,36.385819,46.389874,56.029206
Ours (0.8),15.443813,26.044988,35.205322,44.389510,54.181004
Ours (1.0),15.443813,25.977710,35.988167,44.825366,54.816468
Uniform w/o feedback (0.0),15.443813,28.881957,38.206001,48.787547,57.577906
Uniform w/o feedback (0.2),15.443813,28.881957,38.206001,48.787547,57.577906


## Note: Ours (1.0) AND Uniform+Sampling Sketch (1.0) have the same perfromance becuase both of them only use SVD when r/R=1 !!

In [21]:
per2 = np.zeros((Summaray.shape),dtype=object)
for i in range(Summaray.shape[0]):
    for j in range(Summaray.shape[1]):
        # if Summaray[i,j]>0:
        per2[i,j] = format(Summaray[i,j],'.2f')+'+-'+format(Summaray_var[i,j],'.2f')

In [22]:
# row = ['global','Max Div. Source']+legend0+legend1+legend3+legend4+legend5+legend6+['Random Selection']
df2 = pd.DataFrame(per2, columns = [f'T={i}' for i in range(1,T+1)], index =row)

In [23]:
# df2

In [25]:
df2.drop(legend0, inplace = True)

In [26]:
df2

,T=1,T=2,T=3,T=4,T=5
global,-0.00+-0.00,-0.00+-0.00,-0.00+-0.00,-0.00+-0.00,-0.00+-0.00
Maximum Diveristy,40.20+-11.56,75.97+-19.72,108.64+-25.41,140.32+-29.68,167.85+-31.58
Ours (0.0),15.44+-1.38,27.96+-2.24,37.47+-3.06,47.64+-3.61,58.23+-3.64
Ours (0.2),15.44+-1.38,28.27+-2.81,37.79+-2.50,46.70+-3.34,57.05+-3.76
Ours (0.4),15.44+-1.38,28.48+-2.89,37.43+-2.81,46.93+-4.02,56.68+-4.46
Ours (0.6),15.44+-1.38,27.90+-1.65,36.39+-1.36,46.39+-2.55,56.03+-3.33
Ours (0.8),15.44+-1.38,26.04+-1.07,35.21+-2.53,44.39+-2.59,54.18+-4.04
Ours (1.0),15.44+-1.38,25.98+-2.21,35.99+-1.53,44.83+-2.99,54.82+-1.53
Uniform w/o feedback (0.0),15.44+-1.38,28.88+-1.48,38.21+-1.89,48.79+-2.25,57.58+-1.88
Uniform w/o feedback (0.2),15.44+-1.38,28.88+-1.48,38.21+-1.89,48.79+-2.25,57.58+-1.88


In [1099]:
# df2.to_csv('output/'+out_name+'2.csv')